<a href="https://colab.research.google.com/github/LauraMazzagufo/applicazioniPython/blob/main/generazione_manifest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
from google.colab import files
from PIL import Image

# CONFIG GENERICA
base_url = "https://img.serverchroma.cnr.it/iiif"
manifest_base_url = f"{base_url}/iiif-manifests"
image_base_path = "coverless%2F{journal}%2F{filename}"

# UPLOAD FILE
uploaded = files.upload()

# ORDINA
filenames = sorted(uploaded.keys())

# OTTIENE NOME RIVISTA DAL PRIMO FILE
first_file = filenames[0]
journal_raw = first_file.split("_")[0]
journal = journal_raw.lower().replace(" ", "")
manifest_id = f"{journal}_manifest"
manifest_url = f"{manifest_base_url}/{manifest_id}.json"

# INIZIALIZZA MANIFEST
manifest = {
    "@context": "http://iiif.io/api/presentation/2/context.json",
    "@id": manifest_url,
    "@type": "sc:Manifest",
    "label": f"Rivista '{journal_raw}'",
    "description": f"Rivista '{journal_raw}'",
    "license": "https://creativecommons.org/publicdomain/mark/1.0/",
    "attribution": "ISTC - CNR",
    "sequences": [
        {
            "@type": "sc:Sequence",
            "label": "Sequence 1",
            "canvases": []
        }
    ]
}

# GENERA CANVAS
for idx, filename in enumerate(filenames, start=1):
    img = Image.open(filename)
    width, height = img.size
    base_file = os.path.basename(filename)
    label = f"Pagina {idx} - {base_file}"

    encoded_path = image_base_path.format(journal=journal, filename=base_file)

    canvas_id = f"{base_url}/iiif/canvas/{manifest_id}_{idx}"
    canvas = {
        "@id": canvas_id,
        "@type": "sc:Canvas",
        "label": label,
        "height": height,
        "width": width,
        "thumbnail": {
            "@id": f"{base_url}/3/{encoded_path}/full/200,/0/default.jpg",
            "format": "image/jpeg"
        },
        "images": [
            {
                "@id": f"{canvas_id}/annotation",
                "@type": "oa:Annotation",
                "motivation": "sc:painting",
                "on": canvas_id,
                "resource": {
                    "@id": f"{base_url}/3/{encoded_path}/full/max/0/default.jpg",
                    "@type": "dctypes:Image",
                    "format": "image/jpeg",
                    "height": height,
                    "width": width,
                    "service": {
                        "@id": f"{base_url}/3/{encoded_path}",
                        "profile": "http://iiif.io/api/image/2/level2.json",
                        "@context": "http://iiif.io/api/image/2/context.json"
                    }
                }
            }
        ]
    }

    manifest["sequences"][0]["canvases"].append(canvas)

# SALVA MANIFEST
output_filename = f"{manifest_id}.json"
with open(output_filename, "w") as f:
    json.dump(manifest, f, indent=2)

# SCARICA FILE
files.download(output_filename)
